In [ ]:
!ln -s /user/data/courses_data/clinical_trials proj1_dataset


In [ ]:
import xml.etree.ElementTree as ET
import trec
import pandas as pd
import numpy as np
import pprint as pp
import pickle
from Query import Query


Queries = "./topics-2014_2015-summary.topics"
Qrels = "./qrels-clinical_trials.txt"

with open(Queries, 'r') as queries_reader:
    txt = queries_reader.read()

root = ET.fromstring(txt)

#Dictionary with all queries, pairs (query_num,query)
cases = {}
queryLen = {}

#Read of all queries to dictionary queries
for query in root.iter('TOP'):
    q_num = query.find('NUM').text
    q_title = query.find('TITLE').text
    cases[q_num] = Query(q_num, q_title)
    queryLen[q_num] = len(q_title)

eval = trec.TrecEvaluation(cases, Qrels)

#Store the cases in a file
pickle.dump( cases, open( "./data/cases.p", "wb" ) )
pickle.dump( eval, open( "./data/eval.p", "wb" ) )
pickle.dump( queryLen, open( "./data/queriesLen.p", "wb" ) )
# pp.pprint(cases)

In [ ]:
import xml.etree.ElementTree as ET
import tarfile
import os
import pickle
import numpy as np
from ClinicalTrial import ClinicalTrial
from Query import Query

tar = tarfile.open("../clinicaltrials.gov-16_dec_2015.tgz", "r:gz")

#Dictionary with all clinical trials, pairs (clinical_trial_id,clinical trial)
clinical_trials = {}


#Read of all clinical trials to dictionary clinical_trials
for tarinfo in tar:
    if tarinfo.size > 500:

        txt = tar.extractfile(tarinfo).read().decode("utf-8", "strict")
        root = ET.fromstring(txt)

        judged = False
        for doc_id in root.iter('nct_id'):
            if doc_id.text in eval.judged_docs:
                judged = True

        if not judged:
            continue

        clinical_trial = {}

        for nct_id in root.iter('nct_id'):
            clinical_trial["nct_id"] = nct_id.text

        clinical_trial["brief_title"] = ""
        for brief_title in root.iter('brief_title'):
            clinical_trial["brief_title"] = brief_title.text
    
        description = ""
        for detailed_description in root.iter('detailed_description'):
            for child in detailed_description:
                description+=child.text
        clinical_trial["detailed_description"] = description

        summary = ""
        for brief_summary in root.iter('brief_summary'):
            for child in brief_summary:
                summary+=child.text
        clinical_trial["brief_summary"] = summary.strip()

        clinical_trial["inclusion_criteria"] = ""
        clinical_trial["exclusion_criteria"] = ""
        for criteria in root.iter('criteria'):
            for child in criteria:
                text = child.text.strip()
                text = ''.join(text.splitlines())
                if "Inclusion Criteria" and "Exclusion Criteria" in text:
                    part1 = text.split("Inclusion Criteria")
                    part2 = part1[-1].split("Exclusion Criteria")
                    clinical_trial["inclusion_criteria"] = part2[0]
                    clinical_trial["exclusion_criteria"] = part2[-1]
                elif "Inclusion Criteria"in text and "Exclusion Criteria" not in text:
                    part1 = text.split("Inclusion Criteria")
                    clinical_trial["inclusion_criteria"] = part1[-1]
                elif "Inclusion Criteria" not in text and "Exclusion Criteria" in text:
                    part1 = text.split("Exclusion Criteria")
                    clinical_trial["exclusion_criteria"] = part1[-1]
       

        clinical_trial["phase"] = ""
        for phase in root.iter('phase'):
            clinical_trial["phase"] = phase.text

        clinical_trial["study_type"] = ""
        for study_type in root.iter('study_type'):
            clinical_trial["study_type"] = study_type.text
        
        clinical_trial["study_design"] = ""
        for study_design in root.iter('study_design'):
            clinical_trial["study_design"] = study_design.text
            
        clinical_trial["condition"]= ""
        for condition in root.iter('condition'):
            clinical_trial["condition"] = condition.text
            
        clinical_trial["intervention_name"] = ""
        clinical_trial["intervention_type"] = ""
        for intervention in root.iter('intervention'):
            for child in intervention:
                clinical_trial[child.tag] = child.text.strip()

        clinical_trial["gender"]= ""
        for gender in root.iter('gender'):
            clinical_trial["gender"] = gender.text
        
        clinical_trial["minimum_age"]= ""
        for minimum_age in root.iter('minimum_age'):
            clinical_trial["minimum_age"] = minimum_age.text
    
        clinical_trial["maximum_age"]= ""
        for maximum_age in root.iter('maximum_age'):
            clinical_trial["maximum_age"] = maximum_age.text

        """ clinical_trial["gender"] += " " + clinical_trial["minimum_age"] + " " + clinical_trial["maximum_age"]  #Combine age and gender
        print(clinical_trial["gender"]) """

        clinical_trial["healthy_volunteers"] = ""
        for healthy_volunteers in root.iter('healthy_volunteers'):
            clinical_trial["healthy_volunteers"] = healthy_volunteers.text
        
        clinical_trial["mesh_term"] = ""
        for mesh_term in root.iter('mesh_term'):
            clinical_trial["mesh_term"] = mesh_term.text.strip()

        #Criar a clinical trial
        ct = ClinicalTrial(clinical_trial["nct_id"], clinical_trial["brief_title"],clinical_trial["detailed_description"], clinical_trial["brief_summary"], 
        clinical_trial["inclusion_criteria"], clinical_trial["exclusion_criteria"], clinical_trial["phase"],clinical_trial["study_type"], clinical_trial["study_design"], 
        clinical_trial["condition"], clinical_trial["intervention_name"], clinical_trial["intervention_type"], clinical_trial["gender"], clinical_trial["minimum_age"], 
        clinical_trial["maximum_age"], clinical_trial["healthy_volunteers"], clinical_trial["mesh_term"])

        #Adicionar a lista de trials
        clinical_trials[ct.nct_id] = ct

        
tar.close()

#Store the clinical trials in a file
pickle.dump( clinical_trials, open( "./data/clinical_trials.p", "wb" ) )

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import xml.etree.ElementTree as ET
import tarfile
import os
import pickle
import numpy as np
from ClinicalTrial import ClinicalTrial
from Query import Query
import xml.etree.ElementTree as ET
import trec
import pandas as pd
import pprint as pp

#Load data
clinical_trials = pickle.load( open( "./data/clinical_trials.p", "rb" ) )
cases = pickle.load( open( "./data/cases.p", "rb" ) )
eval = pickle.load( open( "./data/eval.p", "rb" ) )

corpus = {'brief_title' : [], 'detailed_description' : [], 'brief_summary' : [], 'inclusion_criteria' : [],
 'exclusion_criteria' : []}

for ct in clinical_trials.values():
        corpus['brief_title'].append(ct.brief_title)
        corpus['detailed_description'].append(ct.detailed_description)
        corpus['brief_summary'].append(ct.brief_summary) 
        corpus['inclusion_criteria'].append(ct.inclusion_criteria) 
        corpus['exclusion_criteria'].append(ct.exclusion_criteria) 

pickle.dump( corpus, open( "./data/corpus.p", "wb" ) )

In [ ]:
from Utils import get_q_rev_docs

get_q_rev_docs(cases, eval)

In [ ]:
from Utils import get_q_docs

get_q_docs(cases, eval)

In [ ]:
# male -> 30- = 1
# male -> 30-60 = 2
# male -> 60+ = 3
# female -> 30- = 4
# female -> 30-60 = 5
# female -> 60+ = 6
#  30- 7
#  30-60 8
#  60+ -> 9
# c.c 10

import pickle
import Utils 

clinical_trials = pickle.load( open( "./data/clinical_trials.p", "rb" ) )
q_revDocs = pickle.load( open( "./data/q_revDocs.p", "rb" ) )

q_label =  Utils.assignLabels(q_revDocs,clinical_trials)

print("Labels",q_label)

pickle.dump( q_label, open( "./data/q_label.p", "wb" ) )


In [ ]:
from Utils import split_sets 

q_label = pickle.load( open( "./data/q_label.p", "rb" ) )
cases = pickle.load( open( "./data/cases.p", "rb" ) )

split_sets(cases,q_label)

